In [1]:
import ROOT
%jsroot on

import FileReader as fr
import CombinedTemplateFit as ctf
import FemtoAnalysis as fa

Welcome to JupyROOT 6.30/06


In [2]:
# definitions
name_data = "root_input/AnalysisResults_data_203202.root"
name_temps = "root_input/AnalysisResults_mc_203201.root"

pt_bins   = [0.5, 0.625, 0.75, 0.875, 1., 1.125, 1.250, 1.375, 1.5, 1.625, 1.75, 1.875, 2.]
pt_bins   = [0.5, 0.625, 0.75, 0.875, 1., 1.250, 1.5, 1.75, 2.]
pt_bins   = [0.5, 0.625, 0.75, 1., 1.5, 2.2]
fitrange = [0.5, 0.2]
sgnlxy = [0.0105+(0.035/((pt_bins[n]+pt_bins[n+1])/2)**1.1) for n in range(len(pt_bins)-1)]
signal = [sgnlxy, 0.1]
print(signal)

# dim 1 = xy, dim 2 = z
dim = 1
labels = ['', 'xy', 'z']
pair = 'p'
tof = False

_fr = fitrange if type(fitrange) in [int, float] else fitrange[0]
_pair = '' if pair == 'p' else '_ap'
_tof = '_TOF' if tof else ''
_r2pair = 'p' if pair == 'p' else 'ap'

tdir = f"femto-dream-debug-track{_pair}_tempFits{_tof}/"
tdir = f"femto-dream-debug-track{_pair}_tempFits_noIB/"

# names for hDCAxy_ thnSparse templates
namelist0 = ["Primary", "SecondaryDaughterLambda", "SecondaryDaughterSigmaplus", "WrongCollision", "Material", "Fake"]
#namelist0 = ["Primary", "SecondaryDaughterLambda", "SecondaryDaughterSigmaplus", "Material", "Fake"]
namelist1 = ["Primary", "Secondary", "WrongCollision", "Material", "Fake"]
#namelist1 = ["Primary", "Secondary", "Material", "Fake"]
namelist2 = ["Primary", "Secondary", "Flat", "Fake"]


[[0.07640726122081498, 0.0633528087117101, 0.05103770762318663, 0.037882117519202, 0.028290130878174252], 0.1]


In [3]:
# get the purities to fix the fake fraction
fpurities = fr.FileReader("purities_22sampling.root")
hpurities = fpurities.GetHisto(f"h_purities_comb_{_r2pair}")
ctf_fracs = ctf.get_fractions_from_purity(hpurities, pt_bins)

# get data
data_file = fr.FileReader(name_data, tdir + "Tracks")
data = data_file.GetHisto("HighDcorrelator")

# get templates
temps_file = fr.FileReader(name_temps, tdir)
temps = [temps_file.GetHisto(f"hDCAxy_{name}", "Tracks_MC") for name in namelist0]

In [4]:
# merge templates (name of the output histogram, 1st hist, 2nd hist, weight of 1st, weight of 2nd)
temps_sec = ctf.merge_templates_weighted(f"hDCAxy_Secondary", temps[1], temps[2], 0.7, 0.3)
temps_flat = ctf.merge_templates_weighted(f"hDCAxy_Flat", temps[3], temps[4], 0, 0)

dca_temps_merged = [temps[0]] 
dca_temps_merged.append(temps_sec)
dca_temps_merged.extend(temps[3:])

dca_temps_flat = [temps[0]]
dca_temps_flat += [temps_sec]
dca_temps_flat += [temps_flat]
dca_temps_flat += [temps[-1]]

#print(dca_temps_flat)
#print([dca_temps_flat[n].GetName() for n in range(len(dca_temps_flat))])
#print(dca_temps)
#print([temp.GetName() for temp in dca_temps_merged])
#print([temp.ClassName() for temp in dca_temps_merged])

In [5]:
"""
temp_limits: limits for the pt bins and can be set up as
    -> 0 --- fix the value of the init value
    -> 1 --- sets limits to [0, 1]
    -> [up1, up2, ...] --- list of upper limits (0 as lower limit) (if 0/1 then fix value/no limits)
    -> [[low, up], ...] --- list of limits
    If the length of the list is smaller than the pt bins, then the last list entry will be taken for the rest of the bins
"""
fraction_prim = {
    "temp_name":    "Primary", 
    "temp_init":    0.9, 
    "temp_limits":  None,
}
fraction_fake = {
    "temp_name":    "Fake", 
    "temp_init":    ctf_fracs, 
    #"temp_init":    ctf_cnvrt,
    "temp_limits":  0,  
    #"temp_limits":  None,
}
fraction_wc = {
    "temp_name":    "WrongCollision", 
    "temp_init":    None, 
    "temp_limits":  [1]*2 + [0],
}
fraction_mt = {
    "temp_name":    "Material", 
    "temp_init":    0.01, 
    "temp_limits":  None,
}
fraction_sec = {
    "temp_name":    "Secondary", 
    "temp_init":    0.1, 
    "temp_limits":  None,
}
fraction_flat = {
    "temp_name":    "Flat", 
    "temp_init":    None, 
    "temp_limits":  None,
}

rebin = 8
_rebin = f"_rb{rebin}" if rebin else ""
wc = 0
if fraction_wc["temp_limits"][0]:
    wc = 1

rename = f"frxy{fitrange[0]}_frz{fitrange[1]}_srxyF_srz{signal[1]}{_rebin}_{_r2pair}.root"
namelist = namelist1
temps_asdf = dca_temps_merged

flat = 0
if flat == 1:
    rename = f"frxy{fitrange[0]}_frz{fitrange[1]}_sr{signal}{_rebin}_{_r2pair}_flat.root"
    namelist = namelist2
    temps_asdf = dca_temps_flat


#rename = "test.root"

settings_tf = {
        "function":         'tf2d',
        "data":             data,
        #"templates":        dca_temps,
        "templates":        temps_asdf,
        "outDir":           "",
        "newfile":          "recreate",
        "rename":           rename,
        "bins":             pt_bins,
        "rebin":            rebin,
        "tftype":           'dca',
        "namelist":         namelist,
        "temp_fraction":    [fraction_fake, fraction_wc, fraction_sec, fraction_flat, fraction_prim, fraction_mt],
        "fitrange":         fitrange,
        "signalrange":      signal,
        "print":            False,
        "debug":            True,
        }

fa.UFFA(settings_tf)
print("\nDone!\n")

FileSaver: file "TemplateFit2D_frxy0.5_frz0.2_srxyF_srz0.1_rb8_p.root" created!
dca_names:     ['Primary', 'Secondary', 'WrongCollision', 'Material', 'Fake']
pt_range:      [(5, 5), (6, 6), (7, 8), (9, 12), (13, 18)]
fitrange:      [[(-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5)], [(-0.2, 0.2), (-0.2, 0.2), (-0.2, 0.2), (-0.2, 0.2), (-0.2, 0.2)]]
signal_range:  [[(-0.07640726122081498, 0.07640726122081498), (-0.0633528087117101, 0.0633528087117101), (-0.05103770762318663, 0.05103770762318663), (-0.037882117519202, 0.037882117519202), (-0.028290130878174252, 0.028290130878174252)], [(-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1)]]
Primary      0.883768
Secondary    0.045071
WrongCollision 0.023820
Material     0.027118
Fake         0.020223

Done!

 FCN=864310 FROM HESSE     STATUS=OK             23 CALLS         215 TOTAL
                     EDM=6.84043e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   S

Info in <TCanvas::Print>: png file temp2d_canvas_1.png has been created
Info in <TCanvas::Print>: png file temp2d_canvas_2.png has been created
Info in <TCanvas::Print>: png file temp2d_canvas_3.png has been created
Info in <TCanvas::Print>: png file temp2d_canvas_4.png has been created
Info in <TCanvas::Print>: png file temp2d_canvas_5.png has been created
Info in <TCanvas::Print>: png file temp2d_total_chi2-ndf.png has been created
Info in <TCanvas::Print>: png file temp2d_hDCA_fractions.png has been created
